In [ ]:
# STEP 1: Install Required Packages
!pip install transformers accelerate sentence-transformers beautifulsoup4 requests faiss-cpu googlesearch-python --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.0 MB/s eta 0:00:00


In [ ]:
# STEP 2: Load TinyLlama Model (1.1B)
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

llm_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
# STEP 3: Web Scraping Helper
import requests
from bs4 import BeautifulSoup

def scrape_text_from_url(url, max_paragraphs=5):
    try:
        response = requests.get(url, timeout=5)
        soup = BeautifulSoup(response.content, "html.parser")
        paragraphs = soup.find_all("p")
        text = " ".join([p.get_text() for p in paragraphs[:max_paragraphs]])
        return text
    except Exception as e:
        return f"Failed to scrape: {e}"

In [ ]:
# STEP 4: Vector Store with FAISS
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

embedder = SentenceTransformer("all-MiniLM-L6-v2")  # Small & efficient

documents = []
metadatas = []
index = faiss.IndexFlatL2(384)  # 384-dim for MiniLM

def add_to_vector_db(text, source):
    embedding = embedder.encode([text])[0]
    documents.append(text)
    metadatas.append({"source": source})
    index.add(np.array([embedding]))

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# STEP 5: RAG Query Function
def generate_answer(user_query):
    from googlesearch import search
    links = list(search(user_query, num_results=3))

    for url in links:
        content = scrape_text_from_url(url)
        if content and "Failed to scrape" not in content:
            add_to_vector_db(content, url)

    query_vec = embedder.encode([user_query])[0]
    D, I = index.search(np.array([query_vec]), k=2)

    if len(I[0]) == 0:
        return "Sorry, I couldn't find relevant information."

    context = "\n\n".join([documents[i] for i in I[0]])

    prompt = f"Based on the following, answer the question clearly.\n\nContext:\n{context}\n\nQuestion: {user_query}\nAnswer:"
    full_response = llm_pipeline(prompt, max_new_tokens=300, do_sample=True, temperature=0.7)[0]["generated_text"]


    if "Answer:" in full_response:
        answer_part = full_response.split("Answer:")[1]
        clean_answer = answer_part.split("Question:")[0].strip()
    else:
        clean_answer = full_response.strip()

    return f"Question: {user_query}\nAnswer: {clean_answer}"



In [ ]:
# STEP 6: Example Query
query = "Explain the concept of overfitting in machine learning"
print(generate_answer(query))

Question: Explain the concept of overfitting in machine learning
Answer: Overfitting in machine learning refers to creating a model that is similar to the training data but performs poorly on new data. This happens when the model is too similar to the training data, resulting in a model that performs well on the training data but fails on new data. Overfitting occurs when the model learns to memorize the training data and not to generalize well to new data. This can lead to poor performance on unseen data, which can make it challenging to use the model for real-world applications. Overfitting can be mitigated by regularization techniques, such as regularizing the weights of the model to prevent them from being too large or too small.


In [ ]:
query = "What skills do i require to become a data scientist?"
print(generate_answer(query))

Question: What skills do i require to become a data scientist?
Answer: I do not have access to the latest list of online courses available. However, here are some popular and highly rated online courses for data science training. I would advise you to visit the official websites of these courses to check their curriculum, pr


In [ ]:
query = "What is photosynthesis?"
print(generate_answer(query))

Question: What is photosynthesis?
Answer: Photosynthesis is the process by which plants use sunlight, water, and carbon dioxide to create oxygen and energy in the form of sugar.
